In [1]:
import pandas as pd
import matplotlib as plt

In [2]:
# Read the CSV file into a DataFrame
df = pd.read_csv('Hotel Reservations.csv')

# Identify categorical variables that need to be encoded
categorical_columns = ['type_of_meal_plan', 'room_type_reserved', 'market_segment_type', 'booking_status']

# Perform one-hot encoding for each categorical variable
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Display the first few rows of the encoded DataFrame to verify the transformation
print(df_encoded.head())

  Booking_ID  no_of_adults  no_of_children  no_of_weekend_nights  \
0   INN00001             2               0                     1   
1   INN00002             2               0                     2   
2   INN00003             1               0                     2   
3   INN00004             2               0                     0   
4   INN00005             2               0                     1   

   no_of_week_nights  required_car_parking_space  lead_time  arrival_year  \
0                  2                           0        224          2017   
1                  3                           0          5          2018   
2                  1                           0          1          2018   
3                  2                           0        211          2018   
4                  1                           0         48          2018   

   arrival_month  arrival_date  ...  room_type_reserved_Room_Type 3  \
0             10             2  ...                      

In [3]:
print(df.shape)

(36275, 19)


In [7]:
# Check the data types of features in the DataFrame
print(df.dtypes)

Booking_ID                               object
no_of_adults                              int64
no_of_children                            int64
no_of_weekend_nights                      int64
no_of_week_nights                         int64
type_of_meal_plan                        object
required_car_parking_space                int64
room_type_reserved                       object
lead_time                                 int64
arrival_year                              int64
arrival_month                             int64
arrival_date                              int64
market_segment_type                      object
repeated_guest                            int64
no_of_previous_cancellations              int64
no_of_previous_bookings_not_canceled      int64
avg_price_per_room                      float64
no_of_special_requests                    int64
booking_status                           object
dtype: object


In [8]:
import pandas as pd

# Assuming your DataFrame is named df
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns

# Print the list of numerical columns
print(numerical_columns)

Index(['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'required_car_parking_space', 'lead_time',
       'arrival_year', 'arrival_month', 'arrival_date', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests'],
      dtype='object')


In [9]:
numerical_columns = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
                     'no_of_week_nights', 'required_car_parking_space', 'lead_time',
                     'arrival_year', 'arrival_month', 'arrival_date', 'repeated_guest',
                     'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
                     'avg_price_per_room', 'no_of_special_requests']

In [10]:
# Calculate the interquartile range (IQR) for each numerical feature
Q1 = df_encoded[numerical_columns].quantile(0.25)
Q3 = df_encoded[numerical_columns].quantile(0.75)
IQR = Q3 - Q1

# Define a threshold for identifying outliers (e.g., 1.5 times the IQR)
threshold = 1.5

# Identify outliers based on the threshold
outliers = ((df_encoded[numerical_columns] < (Q1 - threshold * IQR)) | (df_encoded[numerical_columns] > (Q3 + threshold * IQR))).any(axis=1)

# Remove outliers from the dataset
df_cleaned = df_encoded[~outliers]

# Display the shape of the cleaned dataset to see how many outliers were removed
print("Shape of cleaned dataset:", df_cleaned.shape)

Shape of cleaned dataset: (16746, 29)


In [15]:
# Extract numerical features from the cleaned dataset
X_numeric = df_cleaned[numerical_columns]

# Convert to binary format: 0 for not canceled, 1 for canceled
y_binary = (df_cleaned['booking_status_Not_Canceled'] != 1).astype(int)

# Now, use the binary target variable 'y_binary' in the RFE
# Initialize the RFE object with the Random Forest classifier and specify the number of features to select
rfe = RFE(estimator=rf_classifier, n_features_to_select=10)

# Fit the RFE object to data
rfe.fit(X_numeric, y_binary)

# Get the selected features
selected_features = X_numeric.columns[rfe.support_]
print("Selected Features:", selected_features)

/Users/vanessalittle/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/vanessalittle/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/vanessalittle/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/vanessalittle/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a fu

Selected Features: Index(['no_of_weekend_nights', 'no_of_week_nights', 'lead_time',
       'arrival_month', 'arrival_date', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests'],
      dtype='object')


In [16]:
# Save the cleaned dataset
df_cleaned.to_csv('cleaned_dataset1.csv', index=False)